# Задание 1.1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [4]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])


def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))


In [5]:
# сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])


6

In [6]:
# конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])


'123'

In [7]:
def f(x, y): return x / y


foldl(f, 1, [1, 2, 3]), \
    foldr(f, 1, [1, 2, 3])


(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [14]:
def foldl2(f, x0, lst):
    return foldr(lambda x, y: lambda t: y(f(t, x)), lambda x: x, lst)(x0)


In [15]:
def foldr2(f, x0, lst):
    return foldl(lambda y, x: lambda t: y(f(x, t)), lambda x: x, lst)(x0)


In [16]:
def f(x, y): return x / y


foldl2(f, 1, [1, 2, 3]), \
    foldr2(f, 1, [1, 2, 3])


(0.16666666666666666, 1.5)

# Задание 1.2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [2]:
from itertools import permutations


def check_inv(a, b):
    b_all = [''.join(p) for p in permutations(b)]
    for x in b_all:
        if x in a:
            return True
    return False


In [3]:
a = 'abcrotm'
b = 'tro'
print(check_inv(a, b))

a = 'abcrdotm'
b = 'tro'
print(check_inv(a, b))


True
False


# Задание 1.3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [56]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value

    def __iter__(self):
        if not self.left and not self.right:
            yield self.value
        if self.left:
            yield from self.left
        if self.right:
            yield from self.right

    def __str__(self):
        return f'{self.value}({self.left},{self.right})'.replace('None', '-').replace('(-,-)', '')

    def __repr__(self):
        return '{}({}, {}, {})'.format(self.__class__.__name__,
                                       repr(self.value),
                                       repr(self.left) if self.left else '-',
                                       repr(self.right) if self.right else '-').replace(', -', '')


tree = Tree(0, Tree(1, Tree(3), Tree(4)),
            Tree(2))

list(tree) == [3, 4, 2]



True

In [58]:
print(tree.__iter__())
print(tree.__str__())
print(tree.__repr__())


<generator object Tree.__iter__ at 0x00000258477BBBC0>
0(1(3,4),2)
Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))


# Задание 1.4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [45]:
def calc(expr):
    value = ''
    stack = []
    sign_stack = []
    brack_stack = []
    i = 0
    while i < len(expr):
        x = expr[i]
        if x == ' ':
            pass
        elif x in '1234567890':
            value += x
        elif x in '+-*':
            if x == '*' and expr[i+1] == '*':
                x = '^'
                i += 1
            if value == '' and x == '-':
                pass
            else:
                stack.append(int(value))
            sign_stack.append(x)
            value = ''
        elif x == '(':
            brack_stack.append(i)
            for j in range(len(expr))[i+1::]:
                if expr[j] == ')':
                    brack_stack.pop()
                    if len(brack_stack) == 0:
                        value = calc(expr[i + 1:j])
                        i = j
                        break
                if expr[j] == '(':
                    brack_stack.append(i)
        i += 1
    stack.append(int(value))
    value = ''
    i = 0
    if len(stack) == len(sign_stack) and sign_stack[0] == '-':
        stack[0] = -stack[0]
        sign_stack.pop(0)
    while len(stack) > 1:
        if '^' in sign_stack:
            if sign_stack[i] == '^':
                stack[i] = stack[i] ** stack[i+1]
                stack.pop(i+1)
                sign_stack.pop(i)
                i -= 1
                if '^' not in sign_stack:
                    i = -1
        elif '*' in sign_stack:
            if sign_stack[i] == '*':
                stack[i] = stack[i] * stack[i+1]
                stack.pop(i+1)
                sign_stack.pop(i)
                i -= 1
                if '*' not in sign_stack:
                    i = -1
        else:
            if sign_stack[i] == '+':
                stack[i] = stack[i] + stack[i+1]
            elif sign_stack[i] == '-':
                stack[i] = stack[i] - stack[i+1]
            stack.pop(i+1)
            sign_stack.pop(i)
            i -= 1
        i += 1
    return stack[0]


calc('2 * (15 - 3 * 4) - 2') == 4


True